# Random Forest Classifier

In [ ]:
import gc
from joblib import dump

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from interpret import set_visualize_provider
from interpret.provider import InlineProvider
from interpret import show
from interpret.blackbox import LimeTabular, ShapKernel, PartialDependence, MorrisSensitivity

from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score, balanced_accuracy_score,  f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
gc.collect()

In [ ]:
set_visualize_provider(InlineProvider())

In [ ]:
df = pd.read_csv("../../../data/combined_subjects.csv")

In [ ]:
df.head()

In [ ]:
df.info()

## Data Preparation

In [ ]:
features = [
    "net_acc_std",
    "net_acc_max",
    "EDA_tonic_mean",
    "EDA_tonic_min",
    "EDA_tonic_max",
    "EDA_smna_mean",
    "EDA_smna_std",
    "EDA_smna_min",
    "EDA_smna_max",
    "EDA_phasic_min",
    "label"
]

In [ ]:
df_feat = df[features]
df_feat.head()

### Merged Amusement

In [ ]:
df_feat_merged_amusement = df_feat.copy()
df_feat_merged_amusement["label"] = df_feat_merged_amusement["label"].replace([0], 1)

In [ ]:
df_feat_merged_amusement["label"].unique()

In [ ]:
y_merged_amusement = np.array(df_feat_merged_amusement.pop('label'))
X_merged_amusement = np.array(df_feat_merged_amusement)

In [ ]:
X_train_merged_amusement, X_test_merged_amusement, y_train_merged_amusement, y_test_merged_amusement = train_test_split(X_merged_amusement, y_merged_amusement, test_size=0.25, random_state=42)

### Dropped Amusement

In [ ]:
df_feat_no_amusement = df_feat[df_feat["label"] != 0]

In [ ]:
df_feat_no_amusement["label"].unique()

In [ ]:
y_no_amusement = np.array(df_feat_no_amusement.pop('label'))
X_no_amusement = np.array(df_feat_no_amusement)

In [ ]:
X_train_no_amusement, X_test_no_amusement, y_train_no_amusement, y_test_no_amusement = train_test_split(X_no_amusement, y_no_amusement, test_size=0.25, random_state=42)

## Training

In [ ]:
parameters = dict(
    n_estimators=(25, 50, 75, 100, 125, 150),
    criterion=("gini", "entropy", "log_loss"),
    max_depth=(2, 3, 5, 7, 9, 11),
    random_state=(42,)
)

In [ ]:
forest = RandomForestClassifier()

### Merged Amusement

In [ ]:
clf_forest_merged_amusement = GridSearchCV(estimator=forest, param_grid=parameters)

In [ ]:
clf_forest_merged_amusement.fit(X_train_merged_amusement, y_train_merged_amusement)

In [ ]:
clf_forest_merged_amusement.best_estimator_

In [ ]:
dump(clf_forest_merged_amusement.best_estimator_, "clf_forest_merged_amusement_top_10_feat.joblib")

### Dropped Amusement

In [ ]:
clf_forest_no_amusement = GridSearchCV(estimator=forest, param_grid=parameters)

In [ ]:
clf_forest_no_amusement.fit(X_train_no_amusement, y_train_no_amusement)

In [ ]:
clf_forest_no_amusement.best_estimator_

In [ ]:
dump(clf_forest_no_amusement.best_estimator_, "clf_forest_no_amusement_top_10_feat.joblib")

## Evaluation

### Merged Amusement

In [ ]:
y_pred_merged_amusement = clf_forest_merged_amusement.predict(X_test_merged_amusement)

In [ ]:
accuracy_score(y_test_merged_amusement, y_pred_merged_amusement)

In [ ]:
balanced_accuracy_score(y_test_merged_amusement, y_pred_merged_amusement)

In [ ]:
f1_score(y_test_merged_amusement, y_pred_merged_amusement)

In [ ]:
precision_score(y_test_merged_amusement, y_pred_merged_amusement)

In [ ]:
recall_score(y_test_merged_amusement, y_pred_merged_amusement)

In [ ]:
cm = confusion_matrix(y_test_merged_amusement, y_pred_merged_amusement, labels=clf_forest_merged_amusement.best_estimator_.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_forest_merged_amusement.best_estimator_.classes_)
disp.plot()
plt.show()

### Dropped Amusement

In [ ]:
y_pred_no_amusement = clf_forest_no_amusement.predict(X_test_no_amusement)

In [ ]:
accuracy_score(y_test_no_amusement, y_pred_no_amusement)

In [ ]:
balanced_accuracy_score(y_test_no_amusement, y_pred_no_amusement)

In [ ]:
f1_score(y_test_no_amusement, y_pred_no_amusement)

In [ ]:
precision_score(y_test_no_amusement, y_pred_no_amusement)

In [ ]:
recall_score(y_test_no_amusement, y_pred_no_amusement)

In [ ]:
cm = confusion_matrix(y_test_no_amusement, y_pred_no_amusement, labels=clf_forest_no_amusement.best_estimator_.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_forest_no_amusement.best_estimator_.classes_)
disp.plot()
plt.show()

## XAI

### Merged Amusement

#### Morris Sensitivity Analysis

In [ ]:
msa = MorrisSensitivity(predict_fn=clf_forest_merged_amusement.best_estimator_.predict, data=X_train_merged_amusement, feature_names=features[:-1])
msa_global = msa.explain_global()
show(msa_global)

#### Shapley Additive Explanations

In [ ]:
shap = ShapKernel(predict_fn=clf_forest_merged_amusement.best_estimator_.predict, data=X_train_merged_amusement, feature_names=features[:-1])
shap_local = shap.explain_local(X_test_merged_amusement[:5], y_test_merged_amusement[:5])
show(shap_local)

#### Local Interpretable Model-agnostic Explanations

In [ ]:
lime = LimeTabular(predict_fn=clf_forest_merged_amusement.best_estimator_.predict, data=X_train_merged_amusement, feature_names=features[:-1])
lime_local = lime.explain_local(X_test_merged_amusement[:5], y_test_merged_amusement[:5])
show(lime_local)

#### Partial Dependence Plot

In [ ]:
pdp = PartialDependence(predict_fn=clf_forest_merged_amusement.best_estimator_.predict, data=X_train_merged_amusement, feature_names=features[:-1])
pdp_global = pdp.explain_global()
show(pdp_global)

### Dropped Amusement

#### Morris Sensitivity Analysis

In [ ]:
msa = MorrisSensitivity(predict_fn=clf_forest_no_amusement.best_estimator_.predict, data=X_train_no_amusement, feature_names=features[:-1])
msa_global = msa.explain_global()
show(msa_global)

#### Shapley Additive Explanations

In [ ]:
shap = ShapKernel(predict_fn=clf_forest_no_amusement.best_estimator_.predict, data=X_train_no_amusement, feature_names=features[:-1])
shap_local = shap.explain_local(X_test_merged_amusement[:5], y_test_merged_amusement[:5])
show(shap_local)

#### Local Interpretable Model-agnostic Explanations

In [ ]:
lime = LimeTabular(predict_fn=clf_forest_no_amusement.best_estimator_.predict, data=X_train_no_amusement, feature_names=features[:-1])
lime_local = lime.explain_local(X_test_merged_amusement[:5], y_test_merged_amusement[:5])
show(lime_local)

#### Partial Dependence Plot

In [ ]:
pdp = PartialDependence(predict_fn=clf_forest_no_amusement.best_estimator_.predict, data=X_train_no_amusement, feature_names=features[:-1])
pdp_global = pdp.explain_global()
show(pdp_global)